In [ ]:
input_test = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

output_test = 2024

In [ ]:
from collections import defaultdict
from copy import deepcopy
import random
import itertools

In [ ]:
#Solution for p1 is relatively clean
#Tons of weird exploratory code for p2
#Actual solution for p2 is mostly manual

def update_state_from_line_gate(state, line):
    words = [w for w in line.split(' ') if len(w) == 3 or w == "OR"]
    k0,k1,k2 = words[0], words[2], words[3]
    if k0 in state and k1 in state:
        op = words[1]
        if op == "OR": state[k2] = state[k0] or state[k1]
        elif op == "AND": state[k2] = state[k0] and state[k1]
        elif op == "XOR": state[k2] = (state[k0] + state[k1]) % 2
        else: raise ValueError
        return True
    
    else: return False

def parse_line_gate(line):
    return [w for w in line.split(' ') if len(w) == 3 or w == "OR"]    

def solve(inp):
    state = {}
    blockstring_inputs, blockstring_gates = inp.split('\n\n')
    for line in blockstring_inputs.split('\n'):
        key,val = line.split(': ')
        state[key] = int(val)
    
    lines_gates = blockstring_gates.split('\n')
    lines_evaluated = set()
    while len(lines_evaluated) < len(lines_gates):
        for line_num,line_gate in enumerate(lines_gates):
            if line_num not in lines_evaluated:
                rv = update_state_from_line_gate(state, line_gate)
                if rv: lines_evaluated.add(line_num)
    tuples_z = sorted([(k,v) for (k,v) in state.items() if k.startswith('z')])
    num_z_bin = "".join([str(v) for (k,v) in tuples_z])[::-1]
    return int(num_z_bin, 2)

def get_output(inp, x, y):
    state = {}
    blockstring_inputs, blockstring_gates = inp.split('\n\n')

    str_num_bins = ["{0:b}".format(a) for a in [x,y]]
    
    for char,str_num_bin in zip("xy", str_num_bins):
        #pad with leading zeroes
        str_num_bin = "0" * (45 - len(str_num_bin)) + str_num_bin
        for bit_n,bit_value in enumerate(str_num_bin[::-1]):
            k = str(bit_n)
            #pad with leading zero
            if len(k) == 1: k = "0" + k
            k = char + k
            state[k] = int(bit_value)
    
    lines_gates = blockstring_gates.split('\n')
    lines_evaluated = set()
    while len(lines_evaluated) < len(lines_gates):
        for line_num,line_gate in enumerate(lines_gates):
            if line_num not in lines_evaluated:
                rv = update_state_from_line_gate(state, line_gate)
                if rv: lines_evaluated.add(line_num)
    tuples_z = sorted([(k,v) for (k,v) in state.items() if k.startswith('z')])
    num_z_bin = "".join([str(v) for (k,v) in tuples_z])[::-1]
    return int(num_z_bin, 2)

def get_reverse_trace(inp, gate):
    blockstring_inputs, blockstring_gates = inp.split('\n\n')

    d = defaultdict(list)
    prev = [gate]
    curr = []
    
    while prev:
        for line in blockstring_gates.split('\n'):
            in0,op,in1,out = parse_line_gate(line)
            if out in prev:
                curr += [in0, in1]
                d[out] = [in0, op, in1]
        prev = curr
        curr = []
    return d

#apparently this finds no erorrs, which is a bit weird
#swaps are perhaps "self-contained" ?
def find_first_error(inp):
    for n,gate in enumerate(get_gates("z", 45)):
        d = get_reverse_trace(inp, gate)
        gates = {item for value in d.values() for item in value}
        for _gate in gates:
            print(n, _gate)
            if _gate.startswith("x") or _gate.startswith("y"):
                print(n,_gate, int(_gate[1:]))
                if int(_gate[1:]) > n:
                    print(n,d)
    

def get_gates(c, n):
    return [c + ("" if i > 9 else "0") + str(i) for i in range(n)]

def get_tests():
    tests = []
    for i in range(1,45):
        tests.append([2**i, 0])
        tests.append([0, 2**i])
        tests.append([2**i, 2**i])
        tests.append([2**(i-1) + 2**i, 2**(i-1) + 2**i])
    return tests

def get_base_network(inp):
    blockstring_inputs, blockstring_gates = inp.split('\n\n')
    d = defaultdict(defaultdict)
    
    for line in blockstring_inputs.split('\n'):
        gate, val = line.split(': ')
        d[gate]["inputs"] = []
        d[gate]["op"] = "input"
    for line in blockstring_gates.split('\n'):
        words = [w for w in line.split(' ') if len(w) == 3 or w == "OR"]
        f0,op,f1,t = words
        for f in [f0,f1]:
            if "outputs" in d[f]: d[f]["outputs"].append(t)
            else: d[f]["outputs"] = [t]
        d[t]["inputs"] = [f0, f1]
        d[t]["op"] = op
        
    return d

def list_swap(l, f, t):
    return [e for e in l if e != f] + [t]

def get_swap_network(d):
    k_swappable = [k for k,v in d.items() if "outputs" in v]
    k1,k2 = random.sample(k_swappable, 2)
    k1_output = random.choice(d[k1]["outputs"])
    k2_output = random.choice(d[k2]["outputs"])
    
    while k1_output == k2_output:
        k1,k2 = random.sample(k_swappable, 2)
        k1_output = random.choice(d[k1]["outputs"])
        k2_output = random.choice(d[k2]["outputs"])

    #correct k1 k2 outputs
    d[k1]["outputs"] = list_swap(d[k1]["outputs"], k1_output, k2_output)
    d[k2]["outputs"] = list_swap(d[k2]["outputs"], k2_output, k1_output)
    #correct inputs for outputs
    d[k1_output]["inputs"] = list_swap(d[k1_output]["inputs"], k1, k2)
    d[k2_output]["inputs"] = list_swap(d[k2_output]["inputs"], k2, k1)
    
    
    return d, k1, k2

def padded_bin_str_from_int(x):
    bin_str = "{0:b}".format(x)
    bin_str = "0" * (45 - len(bin_str)) + bin_str
    return bin_str    

def padded_gate_from_char_int(c,i):
    return c + "0" * (2 - len(str(i))) + str(i)

def eval_gate(d, g, gd, gate_vals):
    i1_key,i2_key = gd["inputs"]
    if i1_key not in gate_vals or i2_key not in gate_vals: return False
    i1_val, i2_val = gate_vals[i1_key], gate_vals[i2_key]
    
    op = gd["op"]
    if op == "AND": o = i1_val and i2_val
    elif op == "OR": o = i1_val or i2_val
    elif op == "XOR": o = (i1_val + i2_val) % 2

    gate_vals[g] = o    
    return True

def eval_network(d, x, y):
    gate_vals = defaultdict()
    
    #read inputs
    for n,bit in enumerate(padded_bin_str_from_int(x)[::-1]):
        gate_vals[padded_gate_from_char_int("x", n)] = int(bit)
    for n,bit in enumerate(padded_bin_str_from_int(y)[::-1]):
        gate_vals[padded_gate_from_char_int("y", n)] = int(bit)
    
    run = True
    while run:
        run = False
        for g,gd in d.items():
            if g not in gate_vals:
                rv = eval_gate(d, g, gd, gate_vals)
                if rv: run = True
    
    tuples_z = sorted([(k,v) for (k,v) in gate_vals.items() if k.startswith('z')])
    num_z_bin = "".join([str(v) for (k,v) in tuples_z])[::-1]
    return int(num_z_bin, 2) == x + y

def test_network(d): return sum([not eval_network(d,x,y) for (x,y) in get_tests()])

def find_good_swap(d, i=1):
    errors_best, gates_best = test_network(d), (None, None)
    print("Original errors: ", errors_best)
    for i in range(i):
        d_new = deepcopy(d)
        d_new, k1, k2 = get_swap_network(deepcopy(d_new))
        errors_new = test_network(d_new)
        if errors_new < errors_best:
            d_best, errors_best, gates_best = d_new, errors_new, (k1,k2)
            print("New errors: ", errors_best, "Swaps: ", gates_best)
    
    return errors_best, gates_best

In [ ]:
def find_gate(d, k1, k2, op):
    for k,v in d.items():
        if k1 in v["inputs"] and k2 in v["inputs"] and v["op"] == op:
            return k
    return False

def find_gate_swap(d, k1, k2, op, swap):
    if k1 in swap: k1 = [s for s in swap if s != k1][0]
    if k2 in swap: k2 = [s for s in swap if s != k2][0]
    return find_gate(d, k1, k2, op)

def discover_gates(d,x,y,c):
    q = [x,y,c]
    gates = [x,y,c]
    for i in range(3): #max depth
        q_new = []
        for k in q:
            if "outputs" in d[k]:
                for g in d[k]["outputs"]:
                    if g not in gates:
                        gates.append(g)
                        q_new.append(g)
        q = q_new
    return gates

def verify_layer(d,x,y,c,z,swap):
    #this assumes a particular structure, doesn't hold in general
    #x,y,c are all "the same"
    #tried permuting, doesn't seem to help
    #maybe this is not it
    #probably something with how we do swapping instead
    xy_xor = find_gate_swap(d, x, y, "XOR", swap)
    if not xy_xor: return False

    xy_xor_c_xor = find_gate_swap(d, xy_xor, c, "XOR", swap)
    if not xy_xor_c_xor: return False
    if xy_xor_c_xor != z: return False
    
    xy_and = find_gate_swap(d, x, y, "AND", swap)
    if not xy_and: return False

    xy_xor_c_and = find_gate_swap(d, xy_xor, c, "AND", swap)
    if not xy_xor_c_and: return False

    c_next = find_gate_swap(d, xy_xor_c_and, xy_and, "OR", swap)
    if not c_next: return False
    return c_next

def verify_layer_general(d,x,y,c,z,swap):
    v1 = verify_layer(d,x,y,c,z,swap)
    v2 = verify_layer(d,x,c,y,z,swap)
    v3 = verify_layer(d,c,y,x,z,swap)
    if v1: return v1
    if v2: return v2
    if v3: return v3
    return False
    

def peel_network(d):
    #i=0, special cased
    c_map = defaultdict(str)
    c_map[0] = find_gate(d, "x00", "y00", "AND")

    swaps = []
    
    for i in range(1, 45):
        x, y, c, z = padded_gate_from_char_int("x", i), padded_gate_from_char_int("y", i), c_map[i-1], padded_gate_from_char_int("z", i)
        gates = discover_gates(d,x,y,c)
        
        c_next = verify_layer(d,x,y,c,z,[])
        if c_next: c_map[i] = c_next
        else:
            for swap in itertools.combinations(gates, 2):
                c_next = verify_layer(d,x,y,c,z,swap)
                if c_next:
                    c_map[i] = c_next
                    swaps.append(swap)
                    break
        if i == 14:
            swaps.append(("z14", "vhm"))
            c_map[i] = "bbw"
        if i == 27:
            swaps.append(("z27", "mps"))
            c_map[i] = "mps"
        if i == 39:
            swaps.append(("z39", "msq"))
            c_map[i] = "cqt"
        print(i, c_map[i], swaps, gates)
    return swaps

def solve_p2(inp):
    d = get_base_network(inp)
    swaps = peel_network(d)
    flat_list_swaps = []
    for swap in swaps:
        for s in swap:
            flat_list_swaps.append(s)
    flat_list_swaps = sorted(flat_list_swaps)
    str_output = ""
    for swap in flat_list_swaps: str_output += "," + swap
    return str_output[1:]


i == 14:

x14		z14 = x14 AND y14 (wrong)
y14		rkm = x14 XOR y14
c13 (ndq)	fgv = (x14 XOR y14) AND c13
		vhm = c13 XOR (x14 XOR y14)
		bbw = vhm OR fgv

bbw should be c14
z14 should be some part of the overflow gate

let's swap z14 with vhm

x14		vhm = x14 AND y14
y14		rkm = x14 XOR y14
c13 (ndq)	fgv = (x14 XOR y14) AND c13
		z14 = c13 XOR (x14 XOR y14)			good current bit
		bbw = (x14 AND y14) OR ((x14 or y14) AND c13)	good overflow bit

Keep going manually then :\

i == 27:

x27		jgq = x AND y
y27		kqw = x XOR y
c26=kqj		mps = kqw XOR kqj = (x XOR y) XOR c26		should be current bit
		snv = kqw AND kqj = (x XOR y) AND c26		path to overflow
		z27 = snv OR jgq				should be overflow bit
swap mps with z27

i == 39

x		trn = x XOR y
y		mgb = x AND y
c=gpm		msq = gpm XOR trn = c XOR (x XOR y)		should be current bit
		z = gpm AND trn = c AND (x XOR y)		path to overflow
		cqt = mgb OR msq = (x AND y) OR (c XOR (x XOR y) what

swap z with msq and we have instead

		z = c XOR (x XOR y) : good			goot current
		msq = c AND (x XOR y)				good path to overflow
		cqt = (x AND y) OR (c AND (x XOR y)		good overflow

In [ ]:
solve_p2(input_full)
#d = get_base_network(input_full)
#peel_network(d)
#fails on i==14; maybe it needs a different gate sequencing, there are many valid ways after all
#no, manual seems to find a good swap, it seems to just be a matter of swap logic being wrong

In [ ]:
#Search seems to be way too slow, so maybe simpler cleaner solution
#We can recursively describe how things must look
#z0 = x0 XOR y0
#o0 = x0 AND y0 (this will have a different name)

#Then, for every layer, we simply have
#zn = (xn XOR yn) XOR (on-1)
#on = [(xn XOR yn) AND (on-1)] XOR [xn AND yn]  

#We can, I guess, write this out as gates as well:
#xyXOR = xn XOR yn
#zn = xyXOR XOR (on-1)
#xyAND = xn AND yn
#mixed = xyXOR AND (on-1)
#on = (xyAND) OR (mixed)

#We already know  layer0 is okay, so we can skip it without specialcasing, we only need to identify o0 = x0 AND y0
#Then we verify that everything from x1,y1,o0 follows this pattern, or finding swaps to restore it
#Then we proceed

In [ ]:
#d = get_base_network(input_full)
#test_network(d)
#find_good_swap(d, 1000)

In [ ]:
#find_first_error(input_full)
#for gate in get_gates("z", 46):
#    print(gate, get_reverse_trace(input_full, gate), '\n\n')
for i in range(45):
    n = 2**i
    o = get_output(input_full, n, 0)
    print(i, o == n, o, n, bin(o))
#Seems to be four rather localized errors, maybe as simple as switched out and overflow
#Main thing being that we can probably do a bruteforce search for out gate swaps that fix these errors
#Try swap: see if less errors on test set; if so, keep this swap

In [ ]:
get_output(input_full, 1, 1)

In [ ]:
assert solve(input_test) == output_test
solve(input_full)

In [ ]:
input_full = """x00: 1
x01: 0
x02: 0
x03: 1
x04: 1
x05: 1
x06: 0
x07: 0
x08: 0
x09: 1
x10: 0
x11: 0
x12: 0
x13: 1
x14: 0
x15: 1
x16: 1
x17: 0
x18: 0
x19: 1
x20: 1
x21: 1
x22: 0
x23: 1
x24: 1
x25: 1
x26: 1
x27: 1
x28: 1
x29: 0
x30: 1
x31: 1
x32: 0
x33: 0
x34: 0
x35: 1
x36: 1
x37: 0
x38: 0
x39: 0
x40: 0
x41: 0
x42: 0
x43: 1
x44: 1
y00: 1
y01: 1
y02: 1
y03: 1
y04: 0
y05: 1
y06: 0
y07: 1
y08: 0
y09: 1
y10: 1
y11: 1
y12: 1
y13: 0
y14: 0
y15: 0
y16: 0
y17: 1
y18: 1
y19: 1
y20: 0
y21: 1
y22: 1
y23: 1
y24: 1
y25: 1
y26: 1
y27: 0
y28: 1
y29: 1
y30: 0
y31: 0
y32: 0
y33: 1
y34: 1
y35: 0
y36: 0
y37: 0
y38: 0
y39: 1
y40: 0
y41: 0
y42: 0
y43: 0
y44: 1

y06 AND x06 -> pqj
bbb XOR qgg -> z42
x10 AND y10 -> gdd
cfj OR ftr -> htg
x36 XOR y36 -> kfb
gbf OR gnt -> hhp
fnj OR bcg -> bdb
tbk XOR vmk -> z03
kqw XOR kqj -> mps
x31 AND y31 -> nqn
pvw OR bmt -> srk
rbs AND rtc -> tcc
fwf AND ptm -> mpd
x18 AND y18 -> pvw
bhb XOR mps -> z28
x37 XOR y37 -> rtc
cgk AND kfb -> ngh
fcn XOR kvk -> z34
snv OR jgq -> z27
y31 XOR x31 -> hvv
y11 XOR x11 -> fwf
mjm OR gdd -> ptm
y43 XOR x43 -> chc
x07 XOR y07 -> bjv
ncs OR frj -> vvc
gpm XOR trn -> msq
x29 XOR y29 -> tsr
fww AND khq -> vmj
wqk XOR srk -> z19
kkb AND rwd -> fnj
pjh OR gww -> wwp
mgb OR msq -> cqt
y14 AND x14 -> z14
tsr AND fhw -> wfp
y24 XOR x24 -> kkb
pqr OR dsj -> gpm
dng OR wrc -> vmk
y41 XOR x41 -> nhn
y42 XOR x42 -> qgg
fwf XOR ptm -> z11
gfn OR jmh -> ndq
cgk XOR kfb -> z36
y15 XOR x15 -> jnw
y01 AND x01 -> pjh
twr AND vck -> nnr
ndn XOR njd -> z44
nnw AND htg -> mpb
rkm AND ndq -> fgv
x11 AND y11 -> pfd
vvc XOR hkm -> z06
kqw AND kqj -> snv
y04 XOR x04 -> tjk
kjv OR ngh -> rbs
tjk AND cpn -> tvf
y33 XOR x33 -> twr
fgs OR wfp -> dgt
x13 XOR y13 -> dmq
x33 AND y33 -> ttm
y35 AND x35 -> qvt
chc AND kdp -> jhf
y26 XOR x26 -> nnw
y40 XOR x40 -> kvb
x15 AND y15 -> rvm
tnb AND qgp -> bnt
fcn AND kvk -> jdk
dsk OR qvt -> cgk
x21 XOR y21 -> hdg
nhn AND wkr -> dcs
x22 AND y22 -> bqc
cqt AND kvb -> mvg
x25 XOR y25 -> gdv
cpn XOR tjk -> z04
rvm OR rrc -> dhq
kvb XOR cqt -> z40
y29 AND x29 -> fgs
gdv AND bdb -> cfj
x00 AND y00 -> fgw
nnw XOR htg -> z26
x04 AND y04 -> qmn
x38 AND y38 -> pqr
x40 AND y40 -> grj
vhm OR fgv -> bbw
jbm AND kfk -> bmt
y25 AND x25 -> ftr
cnq AND dht -> mjm
y05 AND x05 -> frj
x32 XOR y32 -> qgp
wns OR kcq -> fhw
y18 XOR x18 -> kfk
tfr XOR trt -> z05
jbm XOR kfk -> z18
x03 XOR y03 -> tbk
tcc OR vtp -> sfj
nnr OR ttm -> kvk
y23 AND x23 -> vnq
y28 AND x28 -> wns
tdh OR tkq -> pvt
mpd OR pfd -> gfj
srk AND wqk -> tkq
y27 AND x27 -> jgq
dpc AND ttj -> dsk
y17 XOR x17 -> tmc
vdn OR nqn -> tnb
jsd OR qwf -> cnq
jhf OR tvc -> ndn
x09 XOR y09 -> qwf
y39 XOR x39 -> trn
y12 AND x12 -> fsv
bpv OR fsv -> jdr
ttj XOR dpc -> z35
x30 XOR y30 -> dnt
y24 AND x24 -> bcg
jrh XOR krg -> z08
rtc XOR rbs -> z37
tvf OR qmn -> tfr
x28 XOR y28 -> bhb
hds OR jdk -> dpc
wwp AND wmw -> dng
rqb XOR hdg -> z21
trn AND gpm -> z39
jdr AND dmq -> gfn
x38 XOR y38 -> wph
x16 XOR y16 -> jbf
njd AND ndn -> psd
y07 AND x07 -> brk
bqc OR bkw -> fww
x06 XOR y06 -> hkm
gcr XOR pvt -> z20
x09 AND y09 -> cnk
y26 AND x26 -> qdh
x39 AND y39 -> mgb
brk OR fjj -> krg
y08 XOR x08 -> jrh
qjg OR vph -> cpn
x01 XOR y01 -> bgb
bhb AND mps -> kcq
dgt AND dnt -> whw
cnk XOR hhp -> z09
gfj XOR gmr -> z12
dhq AND jbf -> npc
y23 XOR x23 -> khq
gdv XOR bdb -> z25
x14 XOR y14 -> rkm
y22 XOR x22 -> sws
hvv XOR ctm -> z31
pnv OR jcp -> rqb
y37 AND x37 -> vtp
dmq XOR jdr -> z13
y08 AND x08 -> gbf
gmp XOR bjv -> z07
x10 XOR y10 -> dht
bjv AND gmp -> fjj
sws XOR jmq -> z22
sfj XOR wph -> z38
x35 XOR y35 -> ttj
x44 XOR y44 -> njd
x02 XOR y02 -> wmw
qgp XOR tnb -> z32
dgt XOR dnt -> z30
x02 AND y02 -> wrc
y32 AND x32 -> gfr
tsr XOR fhw -> z29
jrh AND krg -> gnt
vmj OR vnq -> rwd
pqj OR wgp -> gmp
vss OR psd -> z45
jmq AND sws -> bkw
jbf XOR dhq -> z16
x30 AND y30 -> spf
hvv AND ctm -> vdn
mpb OR qdh -> kqj
tmc AND nmh -> cpg
wmw XOR wwp -> z02
y00 XOR x00 -> z00
y13 AND x13 -> jmh
x43 AND y43 -> tvc
wkr XOR nhn -> z41
gfr OR bnt -> vck
y16 AND x16 -> fcs
y41 AND x41 -> tkv
y36 AND x36 -> kjv
tfr AND trt -> ncs
sfj AND wph -> dsj
x19 AND y19 -> tdh
cpg OR nbj -> jbm
jnw AND bbw -> rrc
qgg AND bbb -> bwg
bbw XOR jnw -> z15
x05 XOR y05 -> trt
dht XOR cnq -> z10
x21 AND y21 -> dgc
bwg OR ggq -> kdp
x20 AND y20 -> jcp
x42 AND y42 -> ggq
pvt AND gcr -> pnv
fgw XOR bgb -> z01
x34 AND y34 -> hds
y03 AND x03 -> qjg
y44 AND x44 -> vss
rqb AND hdg -> rvh
rvh OR dgc -> jmq
y19 XOR x19 -> wqk
cnk AND hhp -> jsd
ndq XOR rkm -> vhm
vmk AND tbk -> vph
khq XOR fww -> z23
x17 AND y17 -> nbj
y20 XOR x20 -> gcr
fgw AND bgb -> gww
y12 XOR x12 -> gmr
kdp XOR chc -> z43
y27 XOR x27 -> kqw
npc OR fcs -> nmh
grj OR mvg -> wkr
nmh XOR tmc -> z17
gfj AND gmr -> bpv
vvc AND hkm -> wgp
spf OR whw -> ctm
y34 XOR x34 -> fcn
dcs OR tkv -> bbb
kkb XOR rwd -> z24
twr XOR vck -> z33"""